In [4]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

2023-02-12 06:11:51.581615: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-12 06:11:51.582879: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-12 06:11:51.583849: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-12 06:11:51.584625: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-12 06:11:51.585298: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11097147352090211880,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 16149905408
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 15842881770489263802
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

In [5]:
!pip install pyvi

In [6]:
from pyvi import ViTokenizer, ViPosTagger
import numpy as np
import pandas as pd
import re
import string
import gensim
import sklearn
import tensorflow as tf
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.layers import *
from keras.layers import *
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.metrics import sparse_categorical_accuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing

In [3]:
def changeToOther(topic=''):
    top_9_subtopic = csv['Topic'].value_counts()[:9].index.tolist()
    if topic not in top_9_subtopic: return 'Other'
    return topic

In [4]:
# Load dataset
csv = pd.read_csv('/kaggle/input/vne-sub-topic-1/Vne_sub_topic_1.csv')
csv['Topic'] = csv['Topic'].apply(lambda x: changeToOther(x))
csv = csv.sample(frac=1)
csv[csv['Topic'] == 'Other'] = csv[csv['Topic'] == 'Other'][:500]
csv = csv.dropna()
csv['Topic'].value_counts()

Other                      500
the-thao_bong-da           300
giao-duc_tin-tuc           296
suc-khoe_tin-tuc           292
the-thao_cac-mon-khac      292
giai-tri_gioi-sao          271
du-lich_diem-den           264
the-thao_world-cup-2022    259
khoa-hoc_tin-tuc           253
suc-khoe_cac-benh          246
Name: Topic, dtype: int64

In [5]:
dataset = csv[['Content', 'Topic']]
dataset = dataset.sample(frac=1)
datanewscontent = dataset['Content']
labelnews = dataset['Topic']
# x = labelnews.value_counts()
# print(x)

In [6]:
print(datanewscontent[:5])
print(labelnews[:5])

1603    14 đội sẽ đá vòng tròn một lượt tính điểm, rồi...
900     123 phim của 50 quốc gia và vùng lãnh thổ sẽ t...
4127    Hầu hết trường hợp dị ứng thực phẩm có liên qu...
1933    Một CĐV có ảnh hưởng trên mạng xã hội Trung Qu...
4056    Cách bài tập thở bằng mũi, bụng giúp tăng lượn...
Name: Content, dtype: object
1603           the-thao_bong-da
900                       Other
4127          suc-khoe_cac-benh
1933    the-thao_world-cup-2022
4056          suc-khoe_cac-benh
Name: Topic, dtype: object


In [7]:
# Text preprocessing 
def text_preprocessing(text=''):
  text = text.lower()
  # text = re.sub('[0-9]+',' ',text)
  text = text.translate(text.maketrans(string.punctuation, ' '*len(string.punctuation)))
  text = re.sub('\s+', ' ', text)
  return text

In [8]:
datanews = []
for d in datanewscontent:
    e=ViTokenizer.tokenize(text_preprocessing(d))
    datanews.append(e)

In [10]:
datanews[:5]

['14 đội sẽ đá vòng_tròn một lượt tính điểm rồi chia làm hai nhóm đá tranh chức vô_địch và đua trụ hạng ở giải đoạn hai v league 2023 thay_đổi thể_thức này nằm trong dự_kiến kế_hoạch thi_đấu mùa giải bóng_đá chuyên_nghiệp quốc_gia 2023 được công_ty cổ_phần bóng_đá chuyên_nghiệp việt nam vpf công_bố hôm_nay v league 2023 sẽ khai_mạc vào ngày 3 2 và kết_thúc vào ngày 20 8 đây sẽ là mùa giải ngắn nhất lịch_sử kể từ khi giải vđqg chuyên_nghiệp ra_đời năm 2000 ban tổ_chức rút ngắn thời_gian tổ_chức v league 2023 để chuẩn_bị cho việc mùa giải kế_tiếp sẽ chuyển sang đá vắt qua hai năm giống các giải châu âu v league 2023 không thi_đấu vòng_tròn hai lượt tính điểm chọn đội vô_địch như mùa giải năm nay các đội sẽ đá vòng_tròn một lượt tính điểm ở giai_đoạn một sau đó 6 đội đứng trên sẽ đá giai_đoạn hai tranh chức vô_địch 8 đội đứng dưới sẽ đua trụ hạng thể_thức thi_đấu này từng được vpf áp_dụng ở mùa 2020 rồi 2021 mùa bị huỷ sau vòng 12 do covid 19 v league 2023 cũng sẽ tăng số đội lên 14 trước

In [16]:
import pickle
pickle.dump(datanews, open('preprocess_datanews.pkl', 'wb'))

In [7]:
def truncatedvectors(data,n_components=300):
  svd_ngram = TruncatedSVD(n_components=n_components, random_state=42)
  svd_ngram.fit(data)
  return svd_ngram.transform(data)

In [35]:
def truncatedvectors(data, n_components=300):
    svd_ngram = pickle.load(open("/kaggle/working/svd_ngram.pickle", "rb"))
    return svd_ngram.transform(data)

In [16]:
svd_ngram = TruncatedSVD(n_components=300, random_state=42)
svd_ngram.fit(X_data_tfidf_ngram)
pickle.dump(svd_ngram, open('svd_ngram.pickle', 'wb'))
X_data_tfidf_ngram = svd_ngram.transform(X_data_tfidf_ngram)

TF-IDF

In [11]:
def tfidf(data):
  tfidf_vect_ngram = TfidfVectorizer(analyzer='word', max_features=30000, ngram_range=(1, 2))
  tfidf_vect_ngram.fit(data)
  X_data_tfidf_ngram =  tfidf_vect_ngram.transform(data)
  return truncatedvectors(X_data_tfidf_ngram)

In [34]:
def tfidf(data):
    tfidf_vect_ngram = pickle.load(open("/kaggle/working/tfidf.pickle", "rb"))
    X_data_tfidf_ngram = tfidf_vect_ngram.transform(data)
    return truncatedvectors(X_data_tfidf_ngram)

In [14]:
import pickle
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', max_features=30000, ngram_range=(1, 2))
tfidf_vect_ngram.fit(datanews)
pickle.dump(tfidf_vect_ngram, open('tfidf.pickle', 'wb'))
X_data_tfidf_ngram =  tfidf_vect_ngram.transform(datanews)

In [18]:
X_data_tfidf = X_data_tfidf_ngram

In [36]:
X_data_tfidf = tfidf(datanews)

In [38]:
X_data_tfidf[0].shape

(300,)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_data_tfidf, labelnews, test_size=0.2, random_state=42, stratify=labelnews)

Word2Vec

In [ ]:
# pip install gensim==3.8.3

In [ ]:
# from gensim.models import Word2Vec,KeyedVectors 
# import os
# word2vec_model_path ="/kaggle/input/word2vecvn-trained-model/wiki.vi.model.bin"
# model = KeyedVectors.load_word2vec_format(word2vec_model_path,binary=True, unicode_errors='ignore')
# vocab = model.wv.vocab
# wv = model.wv

In [ ]:
# def get_word2vec_data(X):
#     word2vec_data = []
#     for x in X:
#         sentence = []
#         for word in x.split(" "):
#             if word in vocab:
#               sentence=sentence+wv[word].ravel().tolist()
#         word2vec_data.append(sentence)
#     return word2vec_data
    
# def change_to_word2vec(data):
#   data2vec=get_word2vec_data(data)
#   lengthOfdata=[len(data2vec[i]) for i,n in enumerate(data2vec)]
#   for i,n in enumerate(data):
#     if(len(data2vec[i])<max(lengthOfdata)):
#       for j in range(1,(max(lengthOfdata)-len(data2vec[i]))+1):
#         data2vec[i].append(0)
#   return truncatedvectors(np.array(data2vec))

In [ ]:
# len(get_word2vec_data(datanews)[0])

In [ ]:
# X_train_w2v=change_to_word2vec(X_train)
# X_test_w2v=change_to_word2vec(X_test)

SVM

In [34]:
clf=svm.SVC(kernel='linear',C=1000)
clf.fit(X_train, y_train)
val_predictions = clf.predict(X_test)
print(classification_report(y_test, val_predictions))

              precision    recall  f1-score   support

    Doi_song       0.75      0.79      0.77        57
     Du_lich       0.97      0.95      0.96        61
    Giai_tri       0.95      0.93      0.94        59
    Giao_duc       0.97      0.98      0.97        57
    Khoa_hoc       0.98      1.00      0.99        51
   Phap_luat       0.92      0.97      0.95        63
    Suc_khoe       0.87      0.78      0.82        60
    The_thao       1.00      1.00      1.00        63

    accuracy                           0.93       471
   macro avg       0.93      0.93      0.93       471
weighted avg       0.93      0.93      0.93       471



In [35]:
pickle.dump(clf, open('Vi_svm_textclassifier.pkl', 'wb'))

LSTM

In [7]:
#learning_rate: [1e-5,5e-5,1e-4,5e-4,1e-3,5e-3]
opt = Adam(learning_rate=0.001)
def create_lstm_model():
    input_layer = Input(shape=(300,))
    layer = Reshape((10, 30,))(input_layer)
    layer = LSTM(128, activation='relu',dropout=0.5, recurrent_dropout=0.5, return_sequences=True)(layer)
    layer = LSTM(128)(layer)
    layer = Dense(64, activation='relu')(layer)
    layer = Dense(32, activation='relu')(layer)
    output_layer = Dense(10, activation='softmax')(layer)
    classifier = Model(input_layer, output_layer)
    classifier.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return classifier

In [8]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
earlyStopping = EarlyStopping(monitor='loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('Vi_lstm_textclassifier.h5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-3, mode='min')

In [29]:
pd.get_dummies(y_train)

,Other,du-lich_diem-den,giai-tri_gioi-sao,giao-duc_tin-tuc,khoa-hoc_tin-tuc,suc-khoe_cac-benh,suc-khoe_tin-tuc,the-thao_bong-da,the-thao_cac-mon-khac,the-thao_world-cup-2022
4102,0,0,0,0,0,1,0,0,0,0
2741,1,0,0,0,0,0,0,0,0,0
4600,1,0,0,0,0,0,0,0,0,0
3641,0,0,0,0,0,0,1,0,0,0
2035,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
737,0,0,1,0,0,0,0,0,0,0
4303,1,0,0,0,0,0,0,0,0,0
4089,0,0,0,0,0,1,0,0,0,0
403,0,0,0,0,1,0,0,0,0,0


In [9]:
# y_train_dummies = pd.get_dummies(y_train).values
classifier = create_lstm_model()
print(classifier.summary())

2023-02-12 06:12:14.858119: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-12 06:12:14.859302: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-12 06:12:14.860236: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-12 06:12:14.861465: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-12 06:12:14.862363: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 300)]             0         
_________________________________________________________________
reshape (Reshape)            (None, 10, 30)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 10, 128)           81408     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 64)                8256      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330   

In [31]:
classifier.fit(X_train, y_train_dummies, validation_split=0.1, epochs=400, batch_size=32, callbacks=mcp_save)
# classifier.fit(X_train, y_train_dummies, validation_split=0.1, epochs=400, batch_size=32, callbacks=[earlyStopping, mcp_save, reduce_lr_loss])

Epoch 1/400


2022-12-13 13:51:10.687289: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


67/67 [==============================] - 9s 49ms/step - loss: 2.2844 - accuracy: 0.1668 - val_loss: 2.2688 - val_accuracy: 0.1261
Epoch 2/400
67/67 [==============================] - 3s 43ms/step - loss: 2.1661 - accuracy: 0.2051 - val_loss: 1.9454 - val_accuracy: 0.2185
Epoch 3/400
67/67 [==============================] - 3s 43ms/step - loss: 2.0048 - accuracy: 0.2051 - val_loss: 1.7741 - val_accuracy: 0.2269
Epoch 4/400
67/67 [==============================] - 3s 47ms/step - loss: 1.8869 - accuracy: 0.2435 - val_loss: 1.6153 - val_accuracy: 0.2311
Epoch 5/400
67/67 [==============================] - 3s 42ms/step - loss: 1.7646 - accuracy: 0.2682 - val_loss: 1.4742 - val_accuracy: 0.3655
Epoch 6/400
67/67 [==============================] - 3s 43ms/step - loss: 1.6865 - accuracy: 0.3005 - val_loss: 1.3121 - val_accuracy: 0.3866
Epoch 7/400
67/67 [==============================] - 3s 47ms/step - loss: 1.5535 - accuracy: 0.3346 - val_loss: 1.1708 - val_accuracy: 0.4244
Epoch 8/400
67/67 

In [32]:
from keras.models import load_model
classifier = load_model('./Vi_lstm_textclassifier.h5')

In [33]:
# label_encoder = preprocessing.LabelEncoder()
# y_test_encode = label_encoder.fit_transform(y_test)
y_pred = classifier.predict(X_test, batch_size=32, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_pred_txt = []
label = pd.get_dummies(y_train).columns.tolist()
for y_pr in y_pred_bool:
    y_pred_txt.append(label[y_pr])
print(classification_report(y_test, y_pred_txt))

19/19 [==============================] - 0s 4ms/step
                         precision    recall  f1-score   support

                  Other       0.76      0.75      0.75       100
       du-lich_diem-den       0.88      0.92      0.90        53
      giai-tri_gioi-sao       0.96      0.87      0.91        54
       giao-duc_tin-tuc       0.88      0.98      0.93        59
       khoa-hoc_tin-tuc       0.91      0.84      0.88        51
      suc-khoe_cac-benh       0.76      0.92      0.83        49
       suc-khoe_tin-tuc       0.94      0.81      0.87        59
       the-thao_bong-da       0.96      0.88      0.92        60
  the-thao_cac-mon-khac       1.00      0.95      0.97        58
the-thao_world-cup-2022       0.86      0.96      0.91        52

               accuracy                           0.88       595
              macro avg       0.89      0.89      0.89       595
           weighted avg       0.88      0.88      0.88       595

